In [1]:
!nvidia-smi

Mon Dec 28 03:34:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rgh6k21u
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rgh6k21u
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=429c8b028d25a91d70f2d2c62428f55b619a9fd60cc2dfd69c18d3b477fca088
  Stored in directory: /tmp/pip-ephem-wheel-cache-fipmtq0j/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [11]:
## Setting parameters
max_len = 64
batch_size = 30
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [16]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive')


In [17]:
dataset_test2 = nlp.data.TSVDataset("/content/drive/MyDrive/Lec_Capture/_dataNLP/example.tsv", field_indices=[1,2], num_discard_samples=1)
data_test2 = BERTDataset(dataset_test2, 0, 1, tok, max_len, True, False)
test_dataloader2 = torch.utils.data.DataLoader(data_test2, batch_size=batch_size, num_workers=5)


In [18]:
t_total = len(test_dataloader2) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [19]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/myBertModel"))

# model.load_state_dict(torch.load("/content/drive/MyDrive/myBertModel"))


<All keys matched successfully>

In [22]:
test_acc=0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
lable1=label.cpu().numpy()
result=torch.max(out, 1)[1].cpu().numpy()
print('-----------------')
print("test_acc : ",test_acc)
print("label  : ", lable1)
print('result : ', result )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



-----------------
test_acc :  1.0
label  :  [1 0 0 0 0 1 0 0 0]
result :  [1 0 0 0 0 1 0 0 0]


# train

In [23]:
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Lec_Capture/_dataNLP/news.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/naver_movie_review.tsv", field_indices=[1,2], num_discard_samples=1)

In [24]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)


In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=100, num_workers=5)
# for i in train_dataloader:
#   print(i)

In [26]:
num_epochs=50
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.5160740613937378 train acc 0.88

epoch 1 train acc 0.8794444444444444


epoch 2 batch id 1 loss 0.0660933405160904 train acc 0.98

epoch 2 train acc 0.9150925925925927


epoch 3 batch id 1 loss 0.08862999826669693 train acc 0.98

epoch 3 train acc 0.9541203703703705


epoch 4 batch id 1 loss 0.05846380814909935 train acc 0.99

epoch 4 train acc 0.9688425925925925


...

epoch 46 batch id 1 loss 0.00024817706434987485 train acc 1.0

epoch 46 train acc 1.0


epoch 47 batch id 1 loss 0.00023637322010472417 train acc 1.0

epoch 47 train acc 1.0


epoch 48 batch id 1 loss 0.0002219067100668326 train acc 1.0

epoch 48 train acc 1.0


epoch 49 batch id 1 loss 0.00024011782079469413 train acc 1.0

epoch 49 train acc 1.0


epoch 50 batch id 1 loss 0.00023198682174552232 train acc 1.0

epoch 50 train acc 1.0


In [27]:
# Model Save
torch.save(model.state_dict(), "/content/drive/MyDrive/reviewModel")


In [28]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/myBertModel"))


<All keys matched successfully>

In [29]:

test_acc=0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
lable1=label.cpu().numpy()
result=torch.max(out, 1)[1].cpu().numpy()
print('-----------------')
print("test_acc : ",test_acc)
print("label  : ", lable1)
print('result : ', result )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



-----------------
test_acc :  7.8758333333333335
label  :  [0 0 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1
 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1]
result :  [0 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1
 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1]
